In [1]:
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
from torchvision import models

from dataloader import make_train_df, SIIMData, ImageTransform
from trainer import Trainer
%matplotlib inline

In [2]:

size = (400, 400)
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.2)
train = make_train_df()[:64]
train_df, val_df = train_test_split(train, test_size=0.25)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_dataset = SIIMData(df=train_df, transform=ImageTransform(size, mean, std), phase="train")
val_dataset = SIIMData(df=val_df, transform=ImageTransform(size, mean, std), phase="val")

  0%|          | 0/6334 [00:00<?, ?it/s]

In [3]:
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [4]:
use_pretrained = True
model = models.vgg16(pretrained=use_pretrained)
model.classifier[6] = nn.Linear(in_features=4096, out_features=4)

params_to_update = []
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

for name, param in model.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

optimizer = optim.Adam(params=params_to_update, lr=1e-3)

criterion = nn.CrossEntropyLoss()

trainer = Trainer(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model=model,
    criterion=criterion,
    optimizer=optimizer)


classifier.6.weight
classifier.6.bias


In [5]:
num_epochs = 2

train_loss_list = []
train_acc_list = []
valid_loss_list = []
valid_acc_list = []

no_train_loss, no_train_acc = trainer.valid_one_cycle()
print(f"No_train Loss: {no_train_loss} Acc: {no_train_acc}")


for epoch in range(1, num_epochs+1):
    print(f"Epoch {epoch}/{num_epochs}")
    print("-" * 20)
    epoch_train_loss, epoch_train_accuracy = trainer.train_one_cycle()
    train_loss_list.append(epoch_train_loss)
    train_acc_list.append(epoch_train_accuracy)
    print(f"train Loss: {epoch_train_loss} Acc: {epoch_train_accuracy}")
    epoch_valid_loss, epoch_valid_accuracy = trainer.valid_one_cycle()
    valid_loss_list.append(epoch_valid_loss)
    valid_acc_list.append(epoch_valid_accuracy)
    print(f"valid Loss: {epoch_valid_loss} Acc: {epoch_valid_accuracy}")


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/2
--------------------


  0%|          | 0/2 [00:00<?, ?it/s]

train Loss: 1.45186452070872 Acc: 0.20833333333333334


  0%|          | 0/1 [00:00<?, ?it/s]

valid Loss: 0.953246533870697 Acc: 0.625
Epoch 2/2
--------------------


  0%|          | 0/2 [00:00<?, ?it/s]

train Loss: 1.1135173638661702 Acc: 0.5208333333333334


  0%|          | 0/1 [00:00<?, ?it/s]

valid Loss: 0.9314088225364685 Acc: 0.625
